In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import plotly

In [3]:
plotly.offline.init_notebook_mode(connected=True)

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
import ccal

CCAL version 1.1.0 @ /home/kwatme/github/ccal/ccal/__init__.py


In [7]:
import sys

sys.path.insert(0, '/home/kwatme/alex/')

import A_gsea_get_enrichment

In [8]:
np.random.seed(seed=ccal.RANDOM_SEED)

In [9]:
size = 10000

in_ = np.random.choice((0, 1), size, p=(0.99, 0.01))

score = np.sort(np.random.normal(size=size))

score_absolute = np.absolute(score)

In [10]:
in_

array([0, 0, 0, ..., 0, 0, 0])

In [11]:
score

array([-4.16912494, -3.75493895, -3.4117345 , ...,  3.54761929,
        3.73851517,  4.16346151])

In [12]:
score_absolute

array([4.16912494, 3.75493895, 3.4117345 , ..., 3.54761929, 3.73851517,
       4.16346151])

In [ ]:
%%timeit

up = in_ * score

down = 1.0 - in_

up /= up.sum()

down /= down.sum()

m = up - down

m

In [14]:
%%timeit

for i in range(in_.size):
    
    pass

143 µs ± 7.69 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
%%timeit

up = np.full(score_absolute.size, 0.0)

down = np.full(score_absolute.size, -1.0)

for i in range(in_.size):
    
    in_i = in_[i]
    
    if in_i:
        
        up[i] = score_absolute[i]
        
        down[i] = 0
        
up /= up.sum()

down /= down.sum()

m = up - down

m

2.27 ms ± 72.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
genes = pd.read_csv("genes.txt", sep="\t", squeeze=True).sort_values().tolist()

# len(genes)

In [17]:
n_sample = 1

gene_x_sample = pd.DataFrame(
    np.random.normal(size=(len(genes), n_sample)),
    index=genes,
    columns=("Sample {}".format(i) for i in range(n_sample)),
)

# gene_x_sample

In [18]:
n_gene_set = 3

gene_sets = []

for i in range(n_gene_set):

    gene_sets.append(np.random.choice(genes, size=100, replace=False))

gene_sets = pd.DataFrame(
    gene_sets, index=("Gene Set {}".format(i) for i in range(n_gene_set))
)

# gene_sets

In [29]:
a = gene_x_sample.iloc[:, 0]

b = gene_sets.iloc[-2]

%timeit ccal.single_sample_gsea(a, b, plot=False)

9.87 ms ± 77.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
import cProfile

In [34]:
cProfile.run("ccal.single_sample_gsea(a, b, plot=False)", sort="time")

         70 function calls in 0.014 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.008    0.008    0.008    0.008 single_sample_gsea.py:25(<listcomp>)
        1    0.006    0.006    0.006    0.006 {built-in method numpy.array}
        1    0.000    0.000    0.014    0.014 single_sample_gsea.py:6(single_sample_gsea)
        1    0.000    0.000    0.014    0.014 {built-in method builtins.exec}
        1    0.000    0.000    0.006    0.006 numeric.py:469(asarray)
        1    0.000    0.000    0.000    0.000 single_sample_gsea.py:21(<dictcomp>)
       13    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 base.py:1117(__iter__)
        1    0.000    0.000    0.014    0.014 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 common.py:1845(_is_dtype_type)
        4    0.000    0.000    0.000    0.000 managers.py:1549(internal_valu

In [22]:
b = gene_sets.iloc[[-2]]

%timeit A_gsea_get_enrichment.A_gsea_get_enrichment(a, b)

245 ms ± 3.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit ccal.single_sample_gseas_2(a, b)

178 ms ± 26.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
a = gene_x_sample.iloc[:, [0]]

b = gene_sets

%timeit ccal.single_sample_gseas(a, b)

120 ms ± 2.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
a = gene_x_sample.iloc[:, 0]

%timeit A_gsea_get_enrichment.A_gsea_get_enrichment(a, b)

321 ms ± 3.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit ccal.single_sample_gseas_2(a, b)

454 ms ± 9.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
cProfile.run("ccal.single_sample_gseas_2(a, b)", sort="time")

         168 function calls (165 primitive calls) in 0.475 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.472    0.157    0.472    0.157 arraysetops.py:484(in1d)
        1    0.001    0.001    0.475    0.475 single_sample_gseas_2.py:4(single_sample_gseas_2)
        1    0.000    0.000    0.000    0.000 {method 'cumsum' of 'numpy.ndarray' objects}
        4    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.000    0.000    0.000    0.000 {method 'astype' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.472    0.472 shape_base.py:270(apply_along_axis)
        3    0.000    0.000    0.472    0.157 arraysetops.py:601(isin)
        6    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        1    0.000    0.000    0.475    0.475 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 stride_tricks.py:39(as_strided)
       

In [ ]:
raise

In [ ]:
np.zeros(100)

In [ ]:
raise

In [ ]:
gene_score = gene_x_sample.iloc[:, 0]

for normalization_method in ("-0-", "0-1"):

    if normalization_method is None:

        gene_score_ = gene_score

    else:

        gene_score_ = ccal.normalize_series_or_dataframe(
            gene_score, None, normalization_method
        )

    for statistic in ("ks", "auc"):

        ccal.single_sample_gsea(
            gene_score_,
            gene_sets.iloc[0],
            statistic=statistic,
            title="normalization_method: {}<br>statistic: {}".format(
                normalization_method, statistic
            ),
        )

In [ ]:
ccal.single_sample_gseas(gene_x_sample, gene_sets)

In [ ]:
from scipy.stats import pearsonr


def function(x, y):

    return pearsonr(x, y)[0]

In [ ]:
n_sample_per_phenotype = n_sample // 2

phenotypes = np.array((0,) * n_sample_per_phenotype + (1,) * n_sample_per_phenotype)

ccal.gsea(
    gene_x_sample,
    phenotypes,
    gene_sets.iloc[0, :],
    function,
    statistic="ks",
    n_permutation=10,
    permuting="gene",
    gene_score_name="Correlation",
)